In [2]:
import httpx
from urllib.parse import unquote
from bs4 import BeautifulSoup
from datetime import datetime

import os

In [5]:
host = os.environ['LS_URL']
username = os.environ['username']
password = os.environ['passwod']

file_save_path = r'/data'


# $env:LS_URL = 'https://befragungen.isv.uni-stuttgart.de'

KeyError: 'passwod'

In [3]:
headers = {
"Accept-Encoding" : "gzip, deflate, br",
"Accept-Language" : "de,en-US;q=0.7,en;q=0.3",
"Connection" : "keep-alive",
"DNT" : "1",
"Host" : "befragungen.isv.uni-stuttgart.de",
"Sec-Fetch-Dest" : "document",
"Sec-Fetch-Mode" : "navigate",
"Sec-Fetch-Site" : "same-origin",
"User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:103.0) Gecko/20100101 Firefox/103.0"
}

In [4]:
url = r'https://befragungen.isv.uni-stuttgart.de/index.php/admin/authentication/sa/login'

In [5]:
r = httpx.get(url, headers=headers)

In [6]:
unquote(r.cookies['YII_CSRF_TOKEN'])

'NDV5N0llS1E4bEV6TkZ0WmpNdllTaWxmRnNxRnpid3o2fpYJD7sTwrSc9vLYacWqmIEPgcBJq7c9hBaO7NTqPg=='

In [44]:
with httpx.Client() as client:
    r = client.get(url, headers=headers)    
    TOKEN = unquote(r.cookies['YII_CSRF_TOKEN'])
    PHPSESSID = unquote(r.cookies['PHPSESSID'])
    # print(r.cookies)
    form_data = {
    "YII_CSRF_TOKEN": TOKEN,
    "PHPSESSID": PHPSESSID,
    "authMethod": "Authdb",
    "user": "LGeiger",
    "password": "663cÄB8a3b9B,._cf5#3µFV33a1c3e",
    "loginlang": "default",
    "action": "login",
    "width": "1825",
    "login_submit": "login"
    }
    r = client.post(url, data=form_data, headers=headers)
    PHPSESSID = unquote(r.cookies['PHPSESSID'])
    print(r, r.status_code, r.cookies, r.headers)
    r = client.get('https://befragungen.isv.uni-stuttgart.de/index.php/admin/survey/sa/listsurveys', headers=headers)
    print(r, r.status_code, r.cookies, r.headers)
    soup = BeautifulSoup(r.text, "html.parser")
    data = soup.find('table', class_="table-striped table").findAll('td', class_="hidden-xs has-link")
    surveys = []
    for d in data:
        try:
            surveys.append(int(d.text))
        except ValueError:
            pass
    for id in surveys:
        dl_url  = "https://befragungen.isv.uni-stuttgart.de/index.php/admin/export/sa/survey/action/exportarchive/surveyid/{id}".format(id=id)
        r = client.get(dl_url, headers=headers)
        # Save file data to local copy survey_archive_819872.lsa
        with open("survey_archive_" + str(id) + '.lsa', 'wb') as file:
            for chunk in r.iter_bytes(1024):
                file.write(chunk)

<Response [302 Found]> 302 <Cookies[<Cookie PHPSESSID=0061d1d12fcfda7d0f3538012c41e10d for befragungen.isv.uni-stuttgart.de />]> Headers([('server', 'nginx/1.19.10'), ('date', 'Wed, 10 Aug 2022 13:28:09 GMT'), ('content-type', 'text/html; charset=UTF-8'), ('content-length', '0'), ('connection', 'keep-alive'), ('x-powered-by', 'PHP/7.4.30'), ('expires', 'Thu, 19 Nov 1981 08:52:00 GMT'), ('cache-control', 'no-store, no-cache, must-revalidate'), ('pragma', 'no-cache'), ('set-cookie', 'YII_CSRF_TOKEN=deleted; expires=Thu, 01-Jan-1970 00:00:01 GMT; Max-Age=0; path=/; SameSite=Lax'), ('set-cookie', 'YII_CSRF_TOKEN=deleted; expires=Thu, 01-Jan-1970 00:00:01 GMT; Max-Age=0; path=/; SameSite=Lax'), ('set-cookie', 'PHPSESSID=0061d1d12fcfda7d0f3538012c41e10d; path=/; HttpOnly'), ('location', 'https://befragungen.isv.uni-stuttgart.de/index.php/admin/index'), ('strict-transport-security', 'max-age=31536000')])
<Response [200 OK]> 200 <Cookies[<Cookie YII_CSRF_TOKEN=Z2VRdHpwdVQxTl9sdHhQSTlEd0xRRW94N